In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import os

In [4]:
df = pd.read_csv('/content/drive/MyDrive/ExMachina/thread_labeled_2001_to_2015/pages_with_at_least_two_threads_2014_2015.tsv', delimiter='\t')
df_2015 = pd.read_csv('/content/drive/MyDrive/ExMachina/comments_article_2015/comments_article_2015_with_predictions.tsv', delimiter='\t')
df_2014 = pd.read_csv('/content/drive/MyDrive/ExMachina/comments_article_2014/comments_article_2014_with_predictions.tsv', delimiter='\t')
df_2013 = pd.read_csv('/content/drive/MyDrive/ExMachina/comments_article_2013/comments_article_2013_with_predictions.tsv', delimiter='\t')
df_2014_2015 = pd.concat([df_2015, df_2014], ignore_index=True)
df_2013_2014_2015 = pd.concat([df_2013, df_2014_2015], ignore_index=True)

In [5]:
df_2013_2014_2015['timestamp'] = pd.to_datetime(df_2013_2014_2015['timestamp'])
df_2013_2014_2015.sort_values(by=['user_id', 'timestamp'], inplace=True)
df_2013_2014_2015['word_count'] = df_2013_2014_2015['comment'].apply(lambda x: len(x.split()))
df_2013_2014_2015['cumulative_word_count_past_month'] = 0

In [6]:
for user_id in df['user_id'].unique():
    user_comments = df_2013_2014_2015[df_2013_2014_2015['user_id'] == user_id]
    rolling_word_counts = []

    for index, row in user_comments.iterrows():
        one_year_ago = row['timestamp'] - pd.DateOffset(month=1)
        rolling_word_counts = [(time, count) for time, count in rolling_word_counts if time >= one_year_ago]
        df_2013_2014_2015.at[index, 'cumulative_word_count_past_month'] = sum(count for time, count in rolling_word_counts)
        rolling_word_counts.append((row['timestamp'], row['word_count']))

In [7]:
df_new = df.merge(df_2013_2014_2015[['rev_id', 'cumulative_word_count_past_month']], on='rev_id', how='left')

In [ ]:
df_new.to_csv('/content/drive/MyDrive/ExMachina/cumulative_word_count_past_month.tsv', sep='\t', index=False)